In [56]:
import numpy as np
import pandas as pd
from data import Ecoli_TestSize, Haberman_TestSize, Transfution_TestSize, Pima_TestSize, Co_Author_TestSize, Ecoli_CNN
import trainning_of_adaboost as toa
from sklearn.ensemble import AdaBoostClassifier
import adaboost_svm, ImAda_DecisionTree
from sklearn.metrics  import classification_report, precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, f1_score, precision_score
import math
from sklearn.ensemble import AdaBoostClassifier
from datetime import datetime
import csv
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from wsvm.application import Wsvm
from sklearn.svm import SVC
from fuzzy.weight import fuzzy
from keras.models import Sequential 
from keras.layers import Dense,Activation,Dropout 
from keras.utils import np_utils
from keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization

In [57]:
def compute_metrics(y_test,y_pred):
    cm_WSVM = confusion_matrix(y_test, y_pred)
    se = cm_WSVM[1,1]/(cm_WSVM[1,0]+cm_WSVM[1,1])
    sp = cm_WSVM[0,0]/(cm_WSVM[0,0]+cm_WSVM[0,1])
    gmean = math.sqrt(se*sp)
    f1s = f1_score(y_test,y_pred)
    acc = accuracy_score(y_test,y_pred)
    pre = precision_score(y_test,y_pred)
    auc = roc_auc_score(y_test, y_pred)

    return sp, se, gmean, f1s, pre, acc, auc, cm_WSVM

In [58]:
# 1. SVM lib
def svm_lib(X_train, y_train,X_test):
    clf = SVC(probability=True, kernel='linear')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [59]:
# 2. SVM hand
# def svm(C,X_train, y_train,X_test):
#     model = Svm(C)
#     model.fit(X_train, y_train)
#     test_pred = model.predict(X_test)
#     return test_pred

In [60]:
# 3. DecisionTree
from sklearn import tree
def decisiontree(X_train, y_train,X_test):
    clf = tree.DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [61]:
# 4. WSVM
def wsvm(C,X_train, y_train,X_test,distribution_weight=None):
    model = Wsvm(C,distribution_weight)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [62]:
# 5. AdaBoost SVM
# def ada_svm(M, C, X_train, y_train, X_test, theta):
#     w, b, a = adaboost_svm.fit(X_train, y_train, M, C, instance_categorization=False, proposed = False, theta=theta)
#     y_pred = adaboost_svm.predict(X_test, w, b, a, M)
#     return y_pred

In [63]:
# 5. AdaBoost SVM
def ada_svm(X_train, y_train, X_test):
    clf = AdaBoostClassifier(SVC(probability=True,kernel='linear'),n_estimators=100,learning_rate=1.0, algorithm='SAMME')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred


In [64]:
# 6. AdaBoost DecisionTree
def ada_decisiontree(X_train, y_train,X_test):
    clf = AdaBoostClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [65]:
# 7. AdaBoost WSVM
def ada_wsvm(M, C, theta, X_train, y_train,X_test):
    w, b, a, d = toa.fit(X_train, y_train, M, C, instance_categorization=True, proposed_preprocessing = False,proposed_alpha = False, test_something = False, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred, a, d

In [66]:
# 8. IM.AdaBoost-1 WSVM
def imada1_wsvm(M, C, theta, X_train, y_train,X_test):
    w, b, a, d = toa.fit(X_train, y_train, M, C, instance_categorization=True, proposed_preprocessing = True, proposed_alpha = False, test_something = False, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred, a, d

In [67]:
#9. IM.AdaBoost-2 WSVM
def imada2_wsvm(M, C, X_train, y_train,X_test):
    w, b, a, d = toa.fit(X_train, y_train, M, C, instance_categorization = True,proposed_preprocessing = False,proposed_alpha=True,test_something = False)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred, a, d

In [68]:
#10. IM.AdaBoost-12 WSVM
def imada_12_wsvm(M, C, theta, X_train, y_train,X_test):
    w, b, a, d = toa.fit(X_train, y_train, M, C, instance_categorization = True,proposed_preprocessing= True,proposed_alpha=True,test_something = False, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred, a, d

In [69]:
#11. IM.AdaBoost1 + SVM
def imada1_svm(M, C, theta, X_train, y_train,X_test):
    w, b, a, d = toa.fit(X_train, y_train, M, C, instance_categorization = False, proposed_preprocessing= True,proposed_alpha=False,test_something = True, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred, a, d


In [70]:
#12. IM.AdaBoost2 + SVM
def imada2_svm(M, C, X_train, y_train,X_test):
    w, b, a, d = toa.fit(X_train, y_train, M, C, instance_categorization = False,proposed_preprocessing = False,proposed_alpha=True,test_something = True)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred, a, d

In [71]:
#13. IM.AdaBoost12 + SVM
def imada_12_svm(M, C, theta, X_train, y_train,X_test):
    w, b, a, d = toa.fit(X_train, y_train, M, C, instance_categorization = False,proposed_preprocessing= True,proposed_alpha=True,test_something = True, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred, a, d

In [72]:
def imada_1_decisiontree(M,theta,X_train, y_train,X_test):
    clf, a, d = ImAda_DecisionTree.fit(X_train, y_train, M, proposed_preprocessing = True, proposed_alpha = False, theta = theta)
    y_pred = ImAda_DecisionTree.predict(X_test, a, clf)
    return y_pred, a, d

In [73]:
def imada_2_decisiontree(M, X_train, y_train,X_test):
    clf, a, d = ImAda_DecisionTree.fit(X_train, y_train, M, proposed_preprocessing = False, proposed_alpha = True)
    y_pred = ImAda_DecisionTree.predict(X_test, a, clf)
    return y_pred, a, d

In [74]:
def imada_12_decisiontree(M,theta,X_train, y_train,X_test):
    clf, a, d = ImAda_DecisionTree.fit(X_train, y_train, M, proposed_preprocessing = True, proposed_alpha = True, theta = theta)
    y_pred = ImAda_DecisionTree.predict(X_test, a, clf)
    return y_pred, a, d

In [75]:
def compute_weight(X, y,name_method ="actual_hyper_lin", name_function = "exp", beta = None,C = None, gamma = None, u = None, sigma = None):
    method = fuzzy.method()
    function = fuzzy.function()
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    try:
        if name_method == "own_class_center": 
            d = method.own_class_center(X, y)
        elif name_method == "estimated_hyper_lin": # actual_hyper_lin, own_class_center
            d = method.estimated_hyper_lin(X, y)
        elif name_method == "own_class_center_opposite":
            d = method.own_class_center_opposite(X, y)
        elif name_method == 'actual_hyper_lin':
            d = method.actual_hyper_lin(X, y,C = C, gamma = gamma)
        elif name_method == 'own_class_center_divided':
            d = method.own_class_center_divided(X, y)
        elif name_method == "distance_center_own_opposite_tam":
            d_own, d_opp, d_tam = method.distance_center_own_opposite_tam(X,y)
        else:
            print('dont exist method')
        
        if name_function == "lin":
            W = function.lin(d)
        elif name_function == "exp":
            W = function.exp(d, beta)
        elif name_function == "lin_center_own":
            W = function.lin_center_own(d, pos_index,neg_index)
        elif name_function == 'gau':
            W = function.gau(d, u, sigma)
        elif name_function == "func_own_opp_new":
            W = function.func_own_opp_new(d_own,d_opp,pos_index,neg_index,d_tam)
        elif name_function == "func_own_opp_new_v1":
            W = function.func_own_opp_new_v1(d_own,d_opp,pos_index,neg_index,d_tam)
        elif name_function == "func_own_opp_new_v2":
            W = function.func_own_opp_new_v2(d_own,d_opp,pos_index,neg_index,d_tam)
    except Exception as e:
        print('dont exist function')
        print(e)
    # pos_index = np.where(y == 1)[0]
    # neg_index = np.where(y == -1)[0]
    r_pos = 1
    r_neg = len(pos_index)/len(neg_index)
    m = []
    W = np.array(W)
    m = W[pos_index]*r_pos
    m = np.append(m, W[neg_index]*r_neg)
    return m

In [76]:
def fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction):
    if namemethod =="own_class_center_opposite" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_center)
    elif namemethod =="own_class_center" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="own_class_center_divided" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="estimated_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="actual_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_actual)
    else:   
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction)
    return distribution_weight

In [84]:
def neural_network(X_train, train_labels, X_test, test_labels):
    model=Sequential()
    # model.add(Dense(1000,input_dim=7,activation='relu')) #Ecoli
    # model.add(Dense(1000,input_dim=3,activation='relu')) #Haberman
    model.add(Dense(1000,input_dim=4,activation='relu')) #Transfusion
    model.add(Dense(500,activation='relu'))
    model.add(Dense(300,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    # model.fit(X_train,train_labels,validation_data=(X_test,test_labels),batch_size=20,epochs=10,verbose=1)
    model.fit(X_train,train_labels,validation_data=(X_test,test_labels),batch_size=20,epochs=10,verbose=2)
    prediction = model.predict(X_test)
    y_label=np.argmax(test_labels,axis=1)
    predict_label=np.argmax(prediction,axis=1)
    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_label,predict_label)
    return sp, se, gmean, f1s, pre, acc, auc, cm_WSVM


In [78]:
# # ####################################### TEST SIZE SCRIPT - NOT FIND BEST PARAMETERS ################################
# M=5
# C=10000
# theta = 1
# N = 1
# test_size = [0.2]
# dataset = Ecoli_TestSize
# dataset_1 = Ecoli_CNN
# name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
# name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1","func_own_opp_new_v2"]
# beta_center, beta_estimate, beta_actual = 0.3, 0.6, 0.7
# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]
# filepath = f'./Experiment/Data_{filename}_TestSize_{test_size}.csv'
# for n in range(0,N):
#     header = ['Test Size','Name Method','Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Precision','Accuracy','AUC','Ma tran nham lan']
#     data = []
#     print("Lan boc: ", n+1)
#     for testsize in test_size:
#         X_train, y_train, X_test, y_test = dataset.load_data(test_size=testsize)    
#         X_train_1, train_labels, X_test_1, test_labels = dataset_1.load_data(test_size=testsize)   
#         #No 1
#         print("Decision Tree starting...\n")
#         y_pred = decisiontree(X_train, y_train, X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = "Decision Tree"
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 2
#         print("SVM (lib) starting...\n")
#         y_pred = svm_lib(X_train, y_train, X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = "SVM (lib)"
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 3
#         print("WSVM starting...\n")
#         N, d = X_train.shape
#         distribution_weight = np.ones(N)
#         y_pred = wsvm(C,X_train, y_train,X_test,distribution_weight)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = "WSVM"
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])
        
#         #No 4
#         print("ADA_Decision Tree starting...\n")
#         y_pred = ada_decisiontree(X_train, y_train, X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ADA_DSTree'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 5
#         print("ADA_SVM starting...\n")
#         y_pred = ada_svm(X_train, y_train, X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ADA_SVM'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 6
#         print("ADA_WSVM starting...\n")
#         y_pred, a, d = ada_wsvm(M, C, theta, X_train, y_train,X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ADA_WSVM'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 7
#         print("ImADA1_SVM starting...\n")
#         y_pred, a, d = imada1_svm(M, C, theta, X_train, y_train,X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ImADA1_SVM'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 8
#         print("ImADA2_SVM starting...\n")
#         y_pred, a, d = imada2_svm(M, C, X_train, y_train,X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ImADA2_SVM'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 9
#         print("ImADA_12_SVM starting...\n")
#         y_pred, a, d = imada_12_svm(M, C, theta, X_train, y_train,X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ImADA_12_SVM'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 10
#         print("ImADA1_WSVM starting...\n")
#         y_pred, a, d = imada1_wsvm(M, C, theta, X_train, y_train,X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ImADA1_WSVM'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 11
#         print("ImADA2_WSVM starting...\n")
#         y_pred, a, d = imada2_wsvm(M, C, X_train, y_train,X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ImADA2_WSVM'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 12
#         print("ImADA_12_WSVM starting...\n")
#         y_pred, a, d = imada_12_wsvm(M, C, theta, X_train, y_train,X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ImADA_12_WSVM'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])
        
#         #No 13
#         print("ImADA_1_DecisionTree starting...\n")
#         y_pred,a,d = imada_1_decisiontree(M, theta, X_train, y_train,X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ImADA_1_DecisionTree'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 14
#         print("ImADA_2_DecisionTree starting...\n")
#         y_pred,a,d = imada_2_decisiontree(M, X_train, y_train,X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ImADA_2_DecisionTree'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 15
#         print("ImADA_12_DecisionTree starting...\n")
#         y_pred,a,d = imada_12_decisiontree(M, theta, X_train, y_train,X_test)
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#         name = 'ImADA_12_DecisionTree'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #No 16
#         print("CNN starting...\n")
#         sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = neural_network(X_train_1, train_labels, X_test_1, test_labels)
#         name = 'CNN'
#         data.append([testsize,name,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#         #FuzyyWsvm
#         for namemethod in name_method:
#             for namefunction in name_function:
#                 if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                     continue
#                 elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                     continue
#                 elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
#                     continue
#                 elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
#                     continue
#                 elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
#                     continue
#                 elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
#                     continue
#                 elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                     continue
#                 elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                     continue
#                 elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                     continue
#                 elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                     continue
#                 elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                     continue
#                 else:
#                     print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                     distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                     __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                     y_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#                     sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                     data.append([testsize,namemethod,namefunction,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])
            

#     with open(f'./Experiment/Data_{filename}_{time}_TestSize.csv', 'a', encoding='UTF8', newline='') as f1:
#         writer = csv.writer(f1)
#         writer.writerow(header)
#         writer.writerows(data)
    



In [85]:
####################################### TEST SIZE SCRIPT - FIND BEST PARAMETERS ################################
# M=[10,20,30,40,50]
# C=[0.1,10,100,1000,10000]
# theta = [0.5,1,1.5,2,2.5]
from data import Haberman_CNN, Transfution_CNN, Pima_CNN
M=[10,15,20,25]
C=[10,100,1000,10000]
theta = [0.3, 0.5, 0.8, 1.2,1.5]
N = 1
test_size = [0.2]
dataset = Transfution_TestSize
dataset_1 = Transfution_CNN
name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1","func_own_opp_new_v2"]
# beta_center, beta_estimate, beta_actual = 0.3, 0.6, 0.7 #ecoli
# beta_center, beta_estimate, beta_actual = 1, 1, 0.6 #Haberman
beta_center, beta_estimate, beta_actual = 0.5, 0.8, 0.1 #Transfusion
time = datetime.now().strftime("%d%m%Y_%H%M%S")
filename = (str(dataset).split("\\")[-1]).split(".")[0]
filepath = f'./Experiment/Data_{filename}_TestSize.csv'
for n in range(0,N):
    header = ['Test Size','Name Method','Name Function','M','C','theta','SP', 'SE', 'Gmean', 'F1 Score','Precision','Accuracy','AUC','Ma tran nham lan']
    data = []
    print("Lan boc: ", n+1)
    for testsize in test_size:
        X_train, y_train, X_test, y_test = dataset.load_data(test_size=testsize)  
        X_train_1, train_labels, X_test_1, test_labels = dataset_1.load_data(test_size=testsize)        
        #No 1
        print("Decision Tree starting...\n")
        y_pred = decisiontree(X_train, y_train, X_test)
        sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
        name = "Decision Tree"
        m = "None"
        c = "none"
        t = "none"
        data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

        #No 2
        print("SVM (lib) starting...\n")
        y_pred = svm_lib(X_train, y_train, X_test)
        sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
        name = "SVM (lib)"
        m = "None"
        c = "none"
        t = "none"
        data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])
        
        #No 3
        print("ADA_Decision Tree starting...\n")
        y_pred = ada_decisiontree(X_train, y_train, X_test)
        sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
        name = 'ADA_DSTree'
        m = "None"
        c = "none"
        t = "none"
        data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

        #No 4
        print("ADA_SVM starting...\n")
        y_pred = ada_svm(X_train, y_train, X_test)
        sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
        name = 'ADA_SVM'
        m = "None"
        c = "none"
        t = "none"
        data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

        #No 5
        print("CNN starting...\n")
        sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = neural_network(X_train_1, train_labels, X_test_1, test_labels)
        name = 'CNN'
        m = "None"
        c = "none"
        t = "none"
        data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

        for m in M:
            for c in C:
                for t in theta:
                    print(m,c,t)
                    #No 6
                    print("WSVM starting...\n")
                    N, d = X_train.shape
                    distribution_weight = np.ones(N)
                    y_pred = wsvm(c,X_train, y_train,X_test,distribution_weight)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = "WSVM"
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #No 7
                    print("ADA_WSVM starting...\n")
                    y_pred, a, d = ada_wsvm(m, c, t, X_train, y_train,X_test)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = 'ADA_WSVM'
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #No 8
                    print("ImADA_1_DecisionTree starting...\n")
                    y_pred,a,d = imada_1_decisiontree(m, t, X_train, y_train,X_test)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = 'ImADA_1_DecisionTree'
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #No 9
                    print("ImADA_2_DecisionTree starting...\n")
                    y_pred,a,d = imada_2_decisiontree(m, X_train, y_train,X_test)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = 'ImADA_2_DecisionTree'
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #No 10
                    print("ImADA_12_DecisionTree starting...\n")
                    y_pred,a,d = imada_12_decisiontree(m, t, X_train, y_train,X_test)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = 'ImADA_12_DecisionTree'
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #No 11
                    print("ImADA1_SVM starting...\n")
                    y_pred, a, d = imada1_svm(m, c, t, X_train, y_train,X_test)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = 'ImADA1_SVM'
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #No 12
                    print("ImADA2_SVM starting...\n")
                    y_pred, a, d = imada2_svm(m, c, X_train, y_train,X_test)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = 'ImADA2_SVM'
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #No 13
                    print("ImADA_12_SVM starting...\n")
                    y_pred, a, d = imada_12_svm(m, c,t, X_train, y_train,X_test)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = 'ImADA_12_SVM'
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #No 14
                    print("ImADA1_WSVM starting...\n")
                    y_pred, a, d = imada1_wsvm(m, c,t, X_train, y_train,X_test)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = 'ImADA1_WSVM'
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #No 15
                    print("ImADA2_WSVM starting...\n")
                    y_pred, a, d = imada2_wsvm(m, c, X_train, y_train,X_test)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = 'ImADA2_WSVM'
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #No 16
                    print("ImADA_12_WSVM starting...\n")
                    y_pred, a, d = imada_12_wsvm(m, c, t, X_train, y_train,X_test)
                    sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                    name = 'ImADA_12_WSVM'
                    data.append([testsize,name,name,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

                    #FuzyyWsvm
                    for namemethod in name_method:
                        for namefunction in name_function:
                            if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
                                continue
                            elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
                                continue
                            elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
                                continue
                            elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
                                continue
                            elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
                                continue
                            elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
                                continue
                            elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
                                continue
                            elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
                                continue
                            elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
                                continue
                            elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
                                continue
                            elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
                                continue
                            else:
                                print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
                                distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                                __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                                y_pred = wsvm(c,X_train, y_train, X_test, distribution_weight)
                                sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
                                data.append([testsize,namemethod,namefunction,m,c,t,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

    with open(f'./Experiment/Data_{filename}_{time}_TestSize.csv', 'a', encoding='UTF8', newline='') as f1:
        writer = csv.writer(f1)
        writer.writerow(header)
        writer.writerows(data)
    



Lan boc:  1
Decision Tree starting...

SVM (lib) starting...

ADA_Decision Tree starting...

ADA_SVM starting...

CNN starting...

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 1000)              4000      
                                                                 
 dense_21 (Dense)            (None, 500)               500500    
                                                                 
 dense_22 (Dense)            (None, 300)               150300    
                                                                 
 dropout_5 (Dropout)         (None, 300)               0         
                                                                 
 dense_23 (Dense)            (None, 2)                 602       
                                                                 
Total params: 655,402
Trainable params: 655,402
Non-tra

In [80]:
# ####################################### TEST SIZE SCRIPT ################################
# M=10
# C=10000
# theta = 2
# N = 5
# test_size = [0.3]
# new_rate = [1/5]

# dataset = Co_Author

# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]
# filepath = f'./Experiment/Data_{filename}_TestSize.csv'
# for n in range(0,N):
#     header = ['Test Size','IR','Method', 'SP', 'SE', 'Gmean', 'F1 Score','Precision','Accuracy','AUC','Ma tran nham lan']
#     data = []
#     print("Lan boc: ", n+1)
#     for testsize in test_size:
#         for newrate in new_rate:
#             X_train, y_train, X_test, y_test = dataset.load_data(test_size=testsize, new_rate=newrate)
#             with open(f'./Experiment/Data_{filename}_{time}_TestSize.csv', 'a', encoding='UTF8', newline='') as f1:
#                 writer = csv.writer(f1)
#                 writer.writerow(header)

#                 print("ADA_Decision Tree starting...\n")
#                 y_pred = ada_decisiontree(X_train, y_train, X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ADA_DSTree'
#                 data.append([testsize,newrate,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ADA_WSVM starting...\n")
#                 y_pred = ada_wsvm(M, C, theta, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ADA_WSVM'
#                 data.append([testsize,newrate,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA1_WSVM starting...\n")
#                 y_pred = imada1_wsvm(M, C, theta, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA1_WSVM'
#                 data.append([testsize,newrate,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA2_WSVM starting...\n")
#                 y_pred = imada2_wsvm(M, C, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA2_WSVM'
#                 data.append([testsize,newrate,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA_12_WSVM starting...\n")
#                 y_pred = imada_12_wsvm(M, C, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA_12_WSVM'
#                 data.append([testsize,newrate,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])
                
#                 writer.writerows(data)


In [81]:
# ######################## K-FOLD SCRIPT WITHOUT CHANGE RATE #########################

# M=100
# C=10000
# theta = 2
# N = 5
# from data import Ecoli_Kfold, Haberman_KFold, Transfution_Kfold
# dataset = Transfution_Kfold


# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]
# filepath = f'./Experiment/Data_{filename}_Full.csv'

# for n in range(0,N):
#     print("Lan boc: ",n+1)
#     X, y = dataset.load_data()
#     print(X.shape)
#     kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
#     header = ['Time','Fold','Method', 'SP', 'SE', 'Gmean', 'F1 Score','Precision','Accuracy','AUC','Ma tran nham lan']
#     data = []
#     with open(f'./Experiment/Data_{filename}_{time}_KFold.csv', 'a', encoding='UTF8', newline='') as f1:
#         writer = csv.writer(f1)
#         writer.writerow(header)
#         fold = 1
#         for train_index, test_index in kfold_validation.split(X,y):   
#             print("\nFold thu ",fold)            
#             # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
#             # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
#             X_train, y_train = X[train_index], y[train_index]
#             X_test, y_test = X[test_index], y[test_index]
#             print(X_test.shape)
            
#             #Scalling Data
#             sc_X = StandardScaler()
#             X_train = sc_X.fit_transform(X_train)
#             X_test = sc_X.transform(X_test)
#             y_train = np.array(y_train)

#             print("ADA_Decision Tree starting...\n")
#             y_pred = ada_decisiontree(X_train, y_train, X_test)
#             sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#             name = 'ADA_DSTree'
#             data.append([n+1,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#             print("ADA_WSVM starting...\n")
#             y_pred = ada_wsvm(M, C, theta, X_train, y_train,X_test)
#             sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#             name = 'ADA_WSVM'
#             data.append([n+1,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#             print("ImADA1_WSVM starting...\n")
#             y_pred = imada1_wsvm(M, C, theta, X_train, y_train,X_test)
#             sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#             name = 'ImADA1_WSVM'
#             data.append([n+1,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#             print("ImADA2_WSVM starting...\n")
#             y_pred = imada2_wsvm(M, C, X_train, y_train,X_test)
#             sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#             name = 'ImADA2_WSVM'
#             data.append([n+1,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#             print("ImADA_12_WSVM starting...\n")
#             y_pred = imada_12_wsvm(M, C, X_train, y_train,X_test)
#             sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#             name = 'ImADA_12_WSVM'
#             data.append([n+1,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])
                
#             fold = fold + 1
#         writer.writerows(data)


In [82]:
# ######################## K-FOLD SCRIPT WITH CHANGE RATE #########################

# M=10
# C=10000
# theta = 0.2
# N = 5
# new_rate = [1/5,1/7,1/9]
# from data import Co_Author_KF

# dataset = Co_Author_KF

# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]
# filepath = f'./Experiment/Data_{filename}_Full.csv'

# for n in range(0,N):
#     print("Lan boc: ",n+1)
#     for ir in new_rate:
#         X, y = dataset.load_data(ir)
#         print("IR = ", ir)
#         print(X.shape)
#         kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
#         header = ['Time','IR','Fold','Method', 'SP', 'SE', 'Gmean', 'F1 Score','Precision','Accuracy','AUC','Ma tran nham lan']
#         data = []
#         with open(f'./Experiment/Data_{filename}_{time}_KFold.csv', 'a', encoding='UTF8', newline='') as f1:
#             writer = csv.writer(f1)
#             writer.writerow(header)
#             fold = 1
#             for train_index, test_index in kfold_validation.split(X,y):   
#                 print("\nFold thu ",fold)            
#                 # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
#                 # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
#                 X_train, y_train = X[train_index], y[train_index]
#                 X_test, y_test = X[test_index], y[test_index]
#                 print(X_test.shape)
                
#                 #Scalling Data
#                 sc_X = StandardScaler()
#                 X_train = sc_X.fit_transform(X_train)
#                 X_test = sc_X.transform(X_test)
#                 y_train = np.array(y_train)

#                 print("ADA_Decision Tree starting...\n")
#                 y_pred = ada_decisiontree(X_train, y_train, X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ADA_DSTree'
#                 data.append([n+1,ir,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ADA_WSVM starting...\n")
#                 y_pred = ada_wsvm(M, C, theta, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ADA_WSVM'
#                 data.append([n+1,ir,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA1_WSVM starting...\n")
#                 y_pred = imada1_wsvm(M, C, theta, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA1_WSVM'
#                 data.append([n+1,ir,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA2_WSVM starting...\n")
#                 y_pred = imada2_wsvm(M, C, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA2_WSVM'
#                 data.append([n+1,ir,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA_12_WSVM starting...\n")
#                 y_pred = imada_12_wsvm(M, C, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA_12_WSVM'
#                 data.append([n+1,ir,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])
                    
#                 fold = fold + 1
#             writer.writerows(data)
